In [ ]:
import numpy as np
from numpy import isnan
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import plotly.figure_factory as ff
import plotly.express as px

In [ ]:
df = pd.read_csv("Autism-Adult-Data1.csv")
df = df.iloc[: , 1:]

KNN IMPUTER

In [ ]:
X = df.drop(['Class/ASD'], axis=1)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609 entries, 0 to 608
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   id               609 non-null    int64
 1   A1_Score         609 non-null    int64
 2   A2_Score         609 non-null    int64
 3   A3_Score         609 non-null    int64
 4   A4_Score         609 non-null    int64
 5   A5_Score         609 non-null    int64
 6   A6_Score         609 non-null    int64
 7   A7_Score         609 non-null    int64
 8   A8_Score         609 non-null    int64
 9   A9_Score         609 non-null    int64
 10  A10_Score        609 non-null    int64
 11  age              609 non-null    int64
 12  gender           609 non-null    int64
 13  ethnicity        609 non-null    int64
 14  jundice          609 non-null    int64
 15  austim           609 non-null    int64
 16  contry_of_res    609 non-null    int64
 17  used_app_before  609 non-null    int64
 18  result    

In [ ]:
y = df['Class/ASD']

In [ ]:
X

,id,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,...,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation
0,1,1,1,1,1,0,0,1,1,0,...,9,0,9,0,0,57,0,6,0,4
1,2,1,1,0,1,0,0,0,1,0,...,7,1,3,0,1,11,0,5,0,4
2,3,1,1,0,1,1,0,1,1,1,...,10,1,3,1,1,49,0,8,0,2
3,4,1,1,0,1,0,0,1,1,0,...,18,0,9,0,1,57,0,6,0,4
4,6,1,1,1,1,1,0,1,1,1,...,19,1,5,1,0,57,0,9,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,699,1,1,1,1,1,1,1,1,1,...,10,0,6,0,0,5,0,10,0,4
605,700,0,1,0,1,1,0,1,1,1,...,8,0,9,0,0,44,0,7,0,4
606,701,1,0,0,0,0,0,0,1,0,...,17,1,2,0,0,33,0,3,0,2
607,703,1,0,0,1,1,0,1,0,1,...,18,1,7,0,0,40,0,6,0,4


In [ ]:
y

0      0
1      0
2      1
3      0
4      1
      ..
604    1
605    1
606    0
607    0
608    1
Name: Class/ASD, Length: 609, dtype: int64

In [ ]:
def knn_imputer(X, k=5):
    Xtrans = X.copy().to_numpy()

    for i in range(X.shape[0]):
        missing_idx = np.where(np.isnan(Xtrans[i]))[0]
        if missing_idx.size == 0:
            continue
        for idx in missing_idx:
            non_nan_idx = np.where(~np.isnan(Xtrans[:, idx]))[0]
            # Calculate Euclidean distances
            distances = np.linalg.norm(Xtrans[non_nan_idx, :] - Xtrans[i, :], axis=1)
            # Find k nearest neighbors
            nearest_idx = non_nan_idx[np.argsort(distances)[:k]]
            # Impute missing value with mean of k nearest neighbors
            Xtrans[i, idx] = np.nanmean(Xtrans[nearest_idx, idx])

    return Xtrans

In [ ]:
Xtrans = knn_imputer(X)

In [ ]:
Xtrans

array([[  1,   1,   1, ...,   6,   0,   4],
       [  2,   1,   1, ...,   5,   0,   4],
       [  3,   1,   1, ...,   8,   0,   2],
       ...,
       [701,   1,   0, ...,   3,   0,   2],
       [703,   1,   0, ...,   6,   0,   4],
       [704,   1,   0, ...,   8,   0,   4]])

In [ ]:
print('Missing: %d' % sum(isnan(Xtrans).flatten()))

Missing: 0


MODEL BUILDING

In [ ]:
def train_test_split(X, y, test_size=0.2, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)

    n_samples = X.shape[0]
    n_test = int(n_samples * test_size)

    indices = np.arange(n_samples)
    np.random.shuffle(indices)

    test_indices = indices[:n_test]
    train_indices = indices[n_test:]

    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]

    return X_train, X_test, y_train, y_test

# Reset indices if X and y are DataFrames
X.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)

# Assuming X is your features and y is your target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train

,id,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,...,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation
165,179,1,0,1,0,1,0,0,1,0,...,12,1,4,0,0,28,0,5,0,4
503,588,1,1,1,1,1,1,0,1,1,...,4,1,1,0,0,11,0,9,0,4
90,104,1,1,0,0,1,0,1,1,0,...,9,1,0,0,0,26,0,5,0,4
528,615,1,1,0,1,0,0,0,1,0,...,4,0,7,0,0,26,0,5,0,4
446,522,1,1,1,1,1,0,0,1,0,...,10,0,9,0,0,32,0,7,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,299,1,0,0,1,0,0,1,1,0,...,10,0,9,1,0,5,0,5,0,4
9,11,1,1,1,1,1,1,1,1,1,...,16,1,9,0,0,57,0,10,0,3
359,426,1,0,0,0,0,0,1,1,0,...,6,1,8,0,0,55,0,4,0,1
192,206,1,1,1,1,1,1,1,1,1,...,37,1,9,0,0,57,0,10,0,4


In [ ]:
# Resetting index for y_train and y_test if needed
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

class LinearSVM:
    def __init__(self, lr=0.01, epochs=1000, tol=1e-5):
        self.lr = lr
        self.epochs = epochs
        self.tol = tol
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for epoch in range(self.epochs):
            for i in range(n_samples):
                if y[i] * (np.dot(X[i], self.weights) + self.bias) >= 1:
                    self.weights -= self.lr * (2 * 1 / self.epochs * self.weights)
                else:
                    self.weights -= self.lr * (2 * 1 / self.epochs * self.weights - np.dot(X[i], y[i]))
                    self.bias -= self.lr * y[i]

    def predict(self, X):
        return np.sign(np.dot(X, self.weights) + self.bias)

# Train Linear SVM
SVM = LinearSVM()
SVM.fit(X_train, y_train)

# Predictions
predictions = SVM.predict(X_test)

# Evaluation
correct = np.sum(predictions == y_test)
accuracy = correct / len(y_test) * 100
print("*Accuracy score for SVM: ", accuracy, "\n")

# Confusion Matrix
def confusion_matrix(y_true, y_pred):
    classes = np.unique(np.concatenate((y_true, y_pred)))
    n_classes = len(classes)
    class_indices = {classes[i]: i for i in range(n_classes)}

    cm = np.zeros((n_classes, n_classes), dtype=int)
    for i in range(len(y_true)):
        true_class_idx = class_indices[y_true[i]]
        pred_class_idx = class_indices[y_pred[i]]
        cm[true_class_idx, pred_class_idx] += 1

    return cm

# Classification Report
def classification_report(y_true, y_pred):
    n_classes = len(np.unique(y_true))
    report = ""
    for i in range(n_classes):
        true_positive = np.sum((y_true == i) & (y_pred == i))
        false_positive = np.sum((y_true != i) & (y_pred == i))
        false_negative = np.sum((y_true == i) & (y_pred != i))
        precision = true_positive / (true_positive + false_positive)
        recall = true_positive / (true_positive + false_negative)
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
        report += f"Class {i}: Precision={precision:.2f}, Recall={recall:.2f}, F1 Score={f1_score:.2f}\n"
    return report

print("*Classification Report for SVM: ")
print(classification_report(y_test, predictions))

*Accuracy score for SVM:  28.09917355371901 

*Classification Report for SVM: 
Class 0: Precision=nan, Recall=0.00, F1 Score=nan
Class 1: Precision=0.28, Recall=1.00, F1 Score=0.44

